# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 16 2022 8:30AM,246568,10,6971184,"Methapharm, Inc.",Released
1,Sep 16 2022 8:30AM,246569,10,SONSB0001924,"Nextsource Biotechnology, LLC",Released
2,Sep 16 2022 8:30AM,246569,10,SONSB0001939,"Nextsource Biotechnology, LLC",Released
3,Sep 16 2022 8:30AM,246572,10,0085983596,ISDIN Corporation,Released
4,Sep 16 2022 8:30AM,246572,10,0085984120,ISDIN Corporation,Released
5,Sep 16 2022 8:30AM,246572,10,0085984152,ISDIN Corporation,Released
6,Sep 16 2022 8:30AM,246572,10,0085984153,ISDIN Corporation,Released
7,Sep 16 2022 8:30AM,246572,10,0085984158,ISDIN Corporation,Released
8,Sep 16 2022 8:30AM,246572,10,0085984166,ISDIN Corporation,Released
9,Sep 16 2022 8:30AM,246572,10,0085984183,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,246573,Released,8
14,246575,Released,1
15,246576,Released,1
16,246577,Released,1
17,246578,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
246573,NaN,8.0
246575,NaN,1.0
246576,NaN,1.0
246577,NaN,1.0
246578,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
246486,0.0,1.0
246510,0.0,1.0
246517,0.0,1.0
246528,0.0,2.0
246532,1.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
246486,0,1
246510,0,1
246517,0,1
246528,0,2
246532,1,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,246486,0,1
1,246510,0,1
2,246517,0,1
3,246528,0,2
4,246532,1,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,246486,,1
1,246510,,1
2,246517,,1
3,246528,,2
4,246532,1,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc."
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC"
3,Sep 16 2022 8:30AM,246572,10,ISDIN Corporation
11,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation
19,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC"
20,Sep 15 2022 7:05PM,246578,16,Sartorius Stedim Filters Inc.
22,Sep 15 2022 5:38PM,246577,21,"NBTY Global, Inc."
23,Sep 15 2022 5:03PM,246576,16,Sartorius Lab Products and Service
24,Sep 15 2022 3:39PM,246560,22,"NBTY Global, Inc."
25,Sep 15 2022 3:18PM,246557,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",,1
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",,2
2,Sep 16 2022 8:30AM,246572,10,ISDIN Corporation,,8
3,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,,8
4,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",,1
5,Sep 15 2022 7:05PM,246578,16,Sartorius Stedim Filters Inc.,,2
6,Sep 15 2022 5:38PM,246577,21,"NBTY Global, Inc.",,1
7,Sep 15 2022 5:03PM,246576,16,Sartorius Lab Products and Service,,1
8,Sep 15 2022 3:39PM,246560,22,"NBTY Global, Inc.",,1
9,Sep 15 2022 3:18PM,246557,19,"GUSA Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,
2,Sep 16 2022 8:30AM,246572,10,ISDIN Corporation,8,
3,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,
4,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,
5,Sep 15 2022 7:05PM,246578,16,Sartorius Stedim Filters Inc.,2,
6,Sep 15 2022 5:38PM,246577,21,"NBTY Global, Inc.",1,
7,Sep 15 2022 5:03PM,246576,16,Sartorius Lab Products and Service,1,
8,Sep 15 2022 3:39PM,246560,22,"NBTY Global, Inc.",1,
9,Sep 15 2022 3:18PM,246557,19,"GUSA Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,
2,Sep 16 2022 8:30AM,246572,10,ISDIN Corporation,8,
3,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,
4,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,NaN
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,NaN
2,Sep 16 2022 8:30AM,246572,10,ISDIN Corporation,8,NaN
3,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,NaN
4,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Sep 16 2022 8:30AM,246568,10,"Methapharm, Inc.",1,0.0
1,Sep 16 2022 8:30AM,246569,10,"Nextsource Biotechnology, LLC",2,0.0
2,Sep 16 2022 8:30AM,246572,10,ISDIN Corporation,8,0.0
3,Sep 16 2022 8:30AM,246573,10,ISDIN Corporation,8,0.0
4,Sep 16 2022 8:30AM,246575,10,"Nextsource Biotechnology, LLC",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,2465529,45,1.0
16,739671,4,0.0
19,493043,2,0.0
21,246577,1,0.0
22,246560,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,2465529,45,1.0
1,16,739671,4,0.0
2,19,493043,2,0.0
3,21,246577,1,0.0
4,22,246560,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,45,1.0
1,16,4,0.0
2,19,2,0.0
3,21,1,0.0
4,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,45.0
1,16,Released,4.0
2,19,Released,2.0
3,21,Released,1.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21,22
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Released,45.0,4.0,2.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Released,45.0,4.0,2.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Released,45.0,4.0,2.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()